## 20. Attribute Descriptors
A descriptor is a class that implements a protocol consisting of the __get__, __set__, and __delete__ methods. Understanding descriptors is key to Python mastery. 

## LineItem Take #3: A Simple Descriptor
Descriptor class: A class implementing the descriptor protocol. <br>
Managed class: The class where the descriptor instances are declared as class attributes

In [2]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name

    def __set__(self, instance, value):
        ''' self is the descriptor instance, and instance is the managed instance.'''
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError("value must be > 0")


In [4]:
class LineItem:
    weight = Quantity("weight")
    price = Quantity("price")
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price


In [5]:
item = LineItem("toy", 2.5, 100)

In [6]:
item.__dict__

{'description': 'toy', 'weight': 2.5, 'price': 100}

In [8]:
LineItem.__dict__

mappingproxy({'__module__': '__main__',
              'weight': <__main__.Quantity at 0x1eb22c70470>,
              'price': <__main__.Quantity at 0x1eb22c70518>,
              '__init__': <function __main__.LineItem.__init__(self, description, weight, price)>,
              'subtotal': <function __main__.LineItem.subtotal(self)>,
              '__dict__': <attribute '__dict__' of 'LineItem' objects>,
              '__weakref__': <attribute '__weakref__' of 'LineItem' objects>,
              '__doc__': None})

In [9]:
import traceback

try:
    item.weight = 0
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-9-f23604991c7c>", line 4, in <module>
    item.weight = 0
  File "<ipython-input-2-31662b8935ca>", line 9, in __set__
    raise ValueError("value must be > 0")
ValueError: value must be > 0


## LineItem Take #4: Automatic Storage Attribute Names
To avoid retyping the attribute name in the descriptor declarations, we’ll generate a unique string for the storage_name of each Quantity instance. 

In [2]:
class Quantity:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')


In [3]:
class LineItem:
    weight = Quantity()
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price


In [4]:
item1 = LineItem("toy", 1.5, 100)
item2 = LineItem("computer", 2, 300)

In [5]:
item1.__dict__

{'description': 'toy', '_Quantity#0': 1.5, '_Quantity#1': 100}

In [6]:
item2.__dict__

{'description': 'computer', '_Quantity#0': 2, '_Quantity#1': 300}

In [7]:
LineItem.__dict__

mappingproxy({'__module__': '__main__',
              'weight': <__main__.Quantity at 0x21bcbe954e0>,
              'price': <__main__.Quantity at 0x21bcbe95588>,
              '__init__': <function __main__.LineItem.__init__(self, description, weight, price)>,
              'subtotal': <function __main__.LineItem.subtotal(self)>,
              '__dict__': <attribute '__dict__' of 'LineItem' objects>,
              '__weakref__': <attribute '__weakref__' of 'LineItem' objects>,
              '__doc__': None})

In [9]:
LineItem.__dict__["weight"].__dict__

{'storage_name': '_Quantity#0'}

## LineItem Take #5: A New Descriptor Type
We’ll then rewrite Quantity and implement NonBlank by inheriting from Validated and just coding the validate methods.

In [10]:
import abc

class AutoStorage:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
 
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)


In [11]:
class Validated(abc.ABC, AutoStorage):
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    
    @abc.abstractmethod
    def validate(self, instance, value):
        """ return validated value or raise ValueError """


In [12]:
class Quantity(Validated):
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value


class NonBlank(Validated):
    """a string with at least one non-space character"""
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value


We could have these descriptors sitting in a separate module and then import them... here we just use them from within the notebook.

In [14]:
class LineItem:
    description = NonBlank()  # description must be a non-blank string
    weight = Quantity()       # the quantities must be > 0
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price


## Overriding Versus Nonoverriding Descriptors